# Create view in BigQuery

In [10]:
from google.cloud import bigquery
import os
import sys

In [7]:
def load_query_file(filename):
    """
    Load the content of the file `filename` to a string and return it.
    """
    with open(filename, 'r') as f:
        query = f.read()
    return query

In [5]:
def create_view(dataset, view_name, query, 
                credentials='/home/skems/gabinete/projetos/keys-configs/gabinete-compartilhado.json'):
    """
    Create or update view in bigquery.
    
    Input
    -----
    
    dataset : str
        Name of the dataset where the view should be saved.
        
    view_name : str
        Name o the view to be created.
        
    query : str
        Query constituting the view.
        
    credentials : str (default 'gabinete-compartilhado.json')
        Filename of the JSON file constaining GCP credentials.
    """
    
    # Start client:
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials
    client = bigquery.Client()
    
    # Create table/view object:
    dataset_ref = client.dataset(dataset)
    table_ref   = dataset_ref.table(view_name)
    view        = bigquery.Table(table_ref)
    # Set view properties:
    view.view_use_legacy_sql = False
    view.view_query = query

    table_list = [obj.table_id for obj in client.list_tables(dataset)]
    if view_name in table_list:
        client.delete_table(view)
        client.create_table(view, timeout=300)
    else:
        client.create_table(view, timeout=300)

In [8]:
query = load_query_file()

'-- To congresso at 21:22\nSELECT * FROM `gabinete-compartilhado.congresso.camara_deputados_semihomonimos`\n-- To analise_congresso_atividade at 21:24\nSELECT * FROM `gabinete-compartilhado.congresso.camara_deputados_semihomonimos`\n'

In [6]:
create_view('executivo_federal_dou', 'nomeacoes_exoneracoes', query)